
# Problema do Rio: Travessia com IA

Este notebook demonstra, de forma prática e explicativa, como resolver o clássico problema da travessia do rio com um robô, uma galinha, uma raposa e um saco de ração.  
A solução é baseada em conceitos de **espaço de estados**, **transições** e **busca de caminho** — fundamentos importantes para **IA em ambientes estáticos**.

---



## 1. Definindo os Estados

Cada entidade pode estar de um lado do rio: **N** (Near) ou **F** (Far).  
Os elementos são: **Robô, Raposa, Galinha, Ração**.  
Exemplo de estado: `NNNF` → Robô, Raposa e Galinha estão do lado Near, e a Ração no lado Far.

Alguns estados não são permitidos, pois a raposa comeria a galinha ou a galinha comeria a ração sem o robô presente.

Vamos definir os estados válidos e criar representações úteis para o algoritmo.


In [ ]:

from itertools import product

# Gerar todos os estados possíveis (N/F para 4 itens)
all_states = [''.join(s) for s in product('NF', repeat=4)]

def is_valid(state):
    robot, fox, chicken, feed = state
    # Se robô está longe e galinha com raposa → problema
    if robot != chicken and fox == chicken:
        return False
    # Se robô está longe e galinha com ração → problema
    if robot != chicken and chicken == feed:
        return False
    return True

valid_states = [s for s in all_states if is_valid(s)]
valid_states



## 2. Definindo as Transições

O robô pode atravessar sozinho ou levar **um** item (Raposa, Galinha ou Ração).  
Vamos definir uma função para identificar quais transições são válidas entre estados.


In [ ]:

from collections import defaultdict

def get_possible_transitions(state):
    transitions = []
    side = state[0]  # Lado do robô
    for i in range(4):  # 0:robô, 1:raposa, 2:galinha, 3:ração
        if i == 0 or state[i] == side:
            new_state = list(state)
            # Mudar lado do robô e (opcionalmente) de outro item
            new_state[0] = 'F' if side == 'N' else 'N'
            if i != 0:
                new_state[i] = new_state[0]
            new_state = ''.join(new_state)
            if new_state in valid_states:
                transitions.append(new_state)
    return transitions

# Criar grafo de transições
graph = defaultdict(list)
for state in valid_states:
    for target in get_possible_transitions(state):
        graph[state].append(target)

graph['NNNN']  # Mostrando transições do estado inicial



## 3. Buscando a Solução

Agora que temos o grafo de transições, vamos utilizar **Busca em Largura (BFS)** para encontrar o caminho mais curto do estado inicial até o objetivo (`FFFF`).


In [ ]:

from collections import deque

def bfs(start, goal):
    queue = deque([[start]])
    visited = set()

    while queue:
        path = queue.popleft()
        state = path[-1]
        if state == goal:
            return path
        if state in visited:
            continue
        visited.add(state)
        for neighbor in graph[state]:
            new_path = list(path)
            new_path.append(neighbor)
            queue.append(new_path)
    return None

solution_path = bfs('NNNN', 'FFFF')
solution_path



## 4. Visualizando o Caminho da Solução

Vamos mostrar a sequência de passos para resolver o problema da travessia do rio.


In [ ]:

def explain_state(state):
    names = ['Robô', 'Raposa', 'Galinha', 'Ração']
    near = [names[i] for i in range(4) if state[i] == 'N']
    far = [names[i] for i in range(4) if state[i] == 'F']
    return f"📍 Lado NEAR: {', '.join(near):30} | Lado FAR: {', '.join(far)}"

for i, state in enumerate(solution_path):
    print(f"Passo {i}: {state} → {explain_state(state)}")
